#### <center> At scale: Parallelization on your local computers

In [1]:
library(parallel)
for (i in seq(2,16,2)) {
  cl <- makeCluster(i)
  ptm <- proc.time()
  parLapply(cl, 1:10000000,
            function(exponent){
                2^exponent
            })
  run.time <- proc.time() - ptm
  stopCluster(cl)
  print (run.time)
}

   user  system elapsed 
 13.293   0.505  32.036 
   user  system elapsed 
  8.356   0.366  17.817 
   user  system elapsed 
  8.239   0.321  14.694 
   user  system elapsed 
  8.303   0.277  12.728 
   user  system elapsed 
  7.984   0.271  11.672 
   user  system elapsed 
  8.801   0.261  12.100 
   user  system elapsed 
  7.690   0.239  10.523 
   user  system elapsed 
  8.534   0.295  11.015 


#### <center> Streaming data acquisition through Kafka and Python

In [ ]:
library(rkafka)

In [2]:
consumer=rkafka.createConsumer("localhost:2181","Election20160922")
print(rkafka.read(consumer))
rkafka.closeConsumer(consumer)

[1] "Java-Object{com.musigma.consumer.MuConsumer@3db443d5}"
[1] "INFO: Remember to close the consumer after done reading messages"
[1] ""


#### <center> At scale: Big Data analytics on SparkR

In [3]:
if (nchar(Sys.getenv("SPARK_HOME")) < 1) {
  Sys.setenv(SPARK_HOME = "/usr/hdp/current/spark-client")
}

spark_submit_args <- paste("--master yarn-client",
                           "--executor-memory 128g",
                           "--num-executors 40",
                           "--executor-cores 8",
                           paste("--conf spark.executorEnv.PATH=",
                                 paste(Sys.getenv("PATH"),
                                       "/software/anaconda3/2.5.0/envs/R/bin",sep=":"),
                                 sep=""),
                           "sparkr-shell",sep=" ")

Sys.setenv("SPARKR_SUBMIT_ARGS"=spark_submit_args)

In [4]:
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))
sc <- sparkR.init(sparkPackages="com.databricks:spark-csv_2.11:1.2.0")
sqlContext <- sparkRSQL.init(sc)


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var

The following objects are masked from ‘package:base’:

    colnames, colnames<-, endsWith, intersect, rank, rbind, sample,
    startsWith, subset, summary, table, transform



Launching java with spark-submit command /usr/hdp/current/spark-client/bin/spark-submit  --packages com.databricks:spark-csv_2.11:1.2.0 --master yarn-client --executor-memory 128g --num-executors 40 --executor-cores 8 --conf spark.executorEnv.PATH=/software/hdf5/1.10.0/bin:/software/openmpi/1.10.3_gcc610/bin:/software/gcc/5.3.0/bin:/software/zeromq/4.1.5/bin:/software/matlab/R2015a/toolbox/distcomp/bin:/software/matlab/R2015a/bin/glnxa64:/software/matlab/R2015a/bin:/software/anaconda3/2.5.0/bin:/home/lngo/software/jdk1.8.0_91/bin:/home/lngo/software/scala-2.10.4/bin:/usr/lib64/qt-3.3/bin:/bin:/usr/bin:/usr/local/bin:/usr/local/sbin:/usr/sbin:/sbin:/opt/ibutils/bin:/opt/mx/bin:/opt/pbs/default/bin:/opt/pbs/default/sbin:/home/lngo/bin:/software/anaconda3/2.5.0/envs/R/bin sparkr-shell /tmp/RtmpA35oYo/backend_port3c48b252a19 


In [5]:
ptm <- proc.time()
reddit <- read.json(sqlContext, "hdfs:///datasets/reddit_data/*/*.bz2")
proc.time() - ptm

   user  system elapsed 
  0.186   0.151 561.877 

In [6]:
print (printSchema(reddit))

root
 |-- approved_by: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- banned_by: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- mod_reports: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- num_reports: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- removal_reason: string (nullable = true)
 |-- r

In [7]:
ptm <- proc.time()
count (reddit)
proc.time() - ptm

[1] 1659361605

   user  system elapsed 
  0.130   0.104 353.044 

In [8]:
sparkR.stop()